# PINN-SGM Demo: Solving Fokker-Planck for Merton Model

This notebook demonstrates the complete workflow for:
1. Solving the Fokker-Planck equation using PINNs
2. Extracting theoretical score functions
3. Creating hybrid scores for diffusion models

**Author**: Bilal Saleh Husain  
**Course**: FM9561 - Special Topics in Mathematical Finance  
**Institution**: Western University

## 1. Setup and Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Import PINN-SGM components
from pinn_sgm.equations import FokkerPlanckMerton
from pinn_sgm.solvers import PINNSolver
from pinn_sgm.nets import DensityMLP
from pinn_sgm.config import MertonModelConfig, PINNConfig, TrainingConfig, ScoreModelConfig
from pinn_sgm.utils import ScoreExtractor, hybrid_score
from pinn_sgm.utils import plot_density_evolution, plot_score_field, plot_training_history
from pinn_sgm.utils import plot_density_heatmap, plot_error_analysis, plot_3d_surface

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Configure Merton Model

The Merton structural model assumes firm asset value $V_t$ follows:
$$dV_t = \mu V_t dt + \sigma V_t dW_t$$

In log-space $X_t = \ln V_t$:
$$dX_t = \alpha dt + \sigma dW_t, \quad \alpha = \mu - \frac{\sigma^2}{2}$$

In [ ]:
# Configure Merton model parameters
merton_config = MertonModelConfig(
    mu=0.05,       # 5% expected asset return
    sigma=0.2,     # 20% asset volatility
    x0=0.0,        # Initial log-asset value (V_0 = exp(0) = 1)
    debt_threshold=0.8  # Default if V_T < 0.8
)

print(f"Merton Model Configuration:")
print(f"  Asset drift (μ): {merton_config.mu}")
print(f"  Asset volatility (σ): {merton_config.sigma}")
print(f"  Effective drift (α): {merton_config.alpha:.6f}")
print(f"  Initial log-value (X_0): {merton_config.x0}")
print(f"  Debt threshold (D): {merton_config.debt_threshold}")

## 3. Create Fokker-Planck Equation

The density $p(x, t)$ satisfies:
$$\frac{\partial p}{\partial t} + \alpha \frac{\partial p}{\partial x} - \frac{\sigma^2}{2} \frac{\partial^2 p}{\partial x^2} = 0$$

**Analytical Solution** (Gaussian):
$$p(x, t) = \frac{1}{\sqrt{2\pi \sigma^2 t}} \exp\left(-\frac{(x - x_0 - \alpha t)^2}{2\sigma^2 t}\right)$$

In [ ]:
# Create Fokker-Planck equation
equation = FokkerPlanckMerton(
    config=merton_config,
    device=device
)

print(f"\nFokker-Planck Equation: {equation}")

## 4. Configure PINN Solver

The PINN minimizes:
$$\mathcal{L}(\theta) = \mathcal{L}_{\text{PDE}}(\theta) + \mathcal{L}_{\text{IC}}(\theta) + \mathcal{L}_{\text{norm}}(\theta)$$

In [ ]:
# PINN configuration
pinn_config = PINNConfig(
    T=1.0,                     # Terminal time
    x_min=-5.0,                # Spatial domain
    x_max=5.0,
    num_collocation=10000,     # Interior points for PDE
    num_initial=1000,          # Points for initial condition
    num_boundary=0,            # No boundary conditions (unbounded domain)
    device=str(device),
    enforce_normalization=True,
    normalization_weight=1.0
)

# Training configuration
training_config = TrainingConfig(
    batch_size=1024,
    epochs=5000,
    learning_rate=1e-3,
    lr_decay_step=1000,
    lr_decay_rate=0.9,
    optimizer='adam',
    gradient_clip_val=1.0,
    verbose=True,
    log_interval=500
)

print(f"PINN Configuration:")
print(f"  Spatial domain: [{pinn_config.x_min}, {pinn_config.x_max}]")
print(f"  Time horizon: [0, {pinn_config.T}]")
print(f"  Collocation points: {pinn_config.num_collocation}")
print(f"\nTraining Configuration:")
print(f"  Epochs: {training_config.epochs}")
print(f"  Learning rate: {training_config.learning_rate}")
print(f"  Batch size: {training_config.batch_size}")

## 5. Create Neural Network

We use a `DensityMLP` with Softplus output activation to ensure $p(x, t) > 0$:
$$p(x, t) = \text{Softplus}(\text{NN}(x, t)) = \log(1 + e^{z})$$

In [ ]:
# Create neural network
network = DensityMLP(
    input_dim=2,              # (x, t)
    hidden_dims=[64, 64, 64], # 3 hidden layers with 64 neurons each
    activation='tanh',        # Tanh activation for hidden layers
    use_softplus=True         # Softplus output for positivity
)

print(f"\nNeural Network Architecture:")
print(network)
print(f"\nTotal parameters: {sum(p.numel() for p in network.parameters()):,}")

## 6. Train PINN Solver

This cell trains the PINN by minimizing the physics residual and initial condition error.

In [ ]:
# Create PINN solver
solver = PINNSolver(
    equation=equation,
    network=network,
    pinn_config=pinn_config,
    training_config=training_config
)

# Train the PINN
print("\nStarting PINN training...\n")
results = solver.train()

print(f"\n{'='*60}")
print(f"Training completed!")
print(f"Final total loss: {results['final_loss']:.6e}")
print(f"Final PDE loss: {solver.history['loss_pde'][-1]:.6e}")
print(f"Final IC loss: {solver.history['loss_ic'][-1]:.6e}")
print(f"{'='*60}")

## 7. Visualize Training History

In [ ]:
# Plot training history
fig = plot_training_history(solver.history)
plt.show()

## 8. Visualize Density Evolution

Compare PINN solution with analytical Gaussian solution.

In [ ]:
# Plot density evolution at multiple time points
fig = plot_density_evolution(
    network=solver.network,
    x_range=(-5.0, 5.0),
    time_points=[0.1, 0.5, 1.0],
    analytical_solution=equation.analytical_solution,
    num_points=300,
    device=device
)
plt.show()

## 9. 2D Heatmap Visualization

In [ ]:
# Plot 2D heatmap of density evolution
fig = plot_density_heatmap(
    network=solver.network,
    x_range=(-5.0, 5.0),
    t_range=(0.01, 1.0),
    num_x_points=200,
    num_t_points=100,
    device=device
)
plt.show()

## 10. Error Analysis

In [ ]:
# Evaluate error metrics
errors = solver.evaluate_error(num_test_points=1000)

print("\nError Metrics (vs. Analytical Solution):")
print(f"  L2 Error: {errors['l2_error']:.6e}")
print(f"  Max Absolute Error: {errors['max_abs_error']:.6e}")
print(f"  Mean Absolute Error: {errors['mean_abs_error']:.6e}")
print(f"  Mean Relative Error: {errors['mean_rel_error']:.6f}")

# Plot error heatmaps
fig = plot_error_analysis(
    network=solver.network,
    analytical_solution=equation.analytical_solution,
    x_range=(-5.0, 5.0),
    t_range=(0.01, 1.0),
    device=device
)
plt.show()

## 11. Extract Score Function

The **score function** is:
$$s(x, t) = \nabla_x \log p(x, t) = \frac{\nabla_x p(x, t)}{p(x, t)}$$

For the Gaussian analytical solution:
$$s(x, t) = -\frac{x - (x_0 + \alpha t)}{\sigma^2 t}$$

In [ ]:
# Create score extractor
score_extractor = ScoreExtractor(
    network=solver.network,
    device=device,
    epsilon=1e-8
)

# Compute score at test points
x_test = torch.tensor([[-2.0], [0.0], [2.0]], device=device, dtype=torch.float32)
t_test = torch.tensor([[0.5], [0.5], [0.5]], device=device, dtype=torch.float32)

scores = score_extractor(x_test, t_test)

print("\nScore Function Evaluation at t=0.5:")
for i in range(len(x_test)):
    print(f"  s(x={x_test[i].item():.1f}, t=0.5) = {scores[i].item():.6f}")

## 12. Visualize Score Field Evolution

In [ ]:
# Plot score field at multiple time points
fig = plot_score_field(
    score_extractor=score_extractor,
    x_range=(-5.0, 5.0),
    time_points=[0.1, 0.5, 1.0],
    num_points=300,
    device=device
)
plt.show()

## 13. Hybrid Score Field

Blend empirical score (from data) with theoretical score (from PINN):
$$\hat{s}(x, t) = (1 - \phi_t) s_\theta(x, t) + \phi_t s_{\text{theory}}(x, t)$$

In [ ]:
# Configure hybrid scoring
score_config = ScoreModelConfig(
    phi_start=0.0,     # At t=0: use empirical score
    phi_end=1.0,       # At t=T: use theoretical score
    interpolation='linear'
)

# Simulate empirical score (in practice, this would be a trained network)
def mock_empirical_score(x, t):
    """Mock empirical score with some noise."""
    # Add small perturbation to theoretical score
    s_theory = score_extractor(x, t)
    noise = 0.1 * torch.randn_like(s_theory)
    return s_theory + noise

# Compute hybrid scores at different times
x_grid = torch.linspace(-5, 5, 100, device=device).unsqueeze(-1)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, t_val in enumerate([0.1, 0.5, 1.0]):
    t_grid = torch.full((100, 1), t_val, device=device)
    
    # Compute scores
    s_empirical = mock_empirical_score(x_grid, t_grid)
    s_theoretical = score_extractor(x_grid, t_grid)
    s_hybrid = hybrid_score(
        x_grid, t_grid,
        s_empirical, s_theoretical,
        score_config, T=1.0
    )
    
    # Plot
    x_np = x_grid.cpu().numpy().flatten()
    axes[idx].plot(x_np, s_empirical.cpu().numpy(), 'b--', alpha=0.5, label='Empirical')
    axes[idx].plot(x_np, s_theoretical.cpu().numpy(), 'r--', alpha=0.5, label='Theoretical')
    axes[idx].plot(x_np, s_hybrid.cpu().numpy(), 'g-', linewidth=2, label='Hybrid')
    axes[idx].set_xlabel('x')
    axes[idx].set_ylabel('Score')
    axes[idx].set_title(f't = {t_val:.1f} (φ = {score_config.get_phi(t_val, 1.0):.2f})')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nHybrid Score Weights:")
for t in [0.0, 0.25, 0.5, 0.75, 1.0]:
    phi = score_config.get_phi(t, T=1.0)
    print(f"  t={t:.2f}: φ={phi:.3f} (empirical weight: {1-phi:.3f}, theoretical weight: {phi:.3f})")

## 14. Default Probability Calculation

For the Merton model with debt threshold $D$:
$$P(\text{default}) = P(V_T < D) = P(X_T < \ln D) = \Phi\left(\frac{\ln D - (x_0 + \alpha T)}{\sigma\sqrt{T}}\right)$$

In [ ]:
# Compute default probability
T_horizon = 1.0
prob_default = equation.default_probability(t=T_horizon)

print(f"\nDefault Probability Analysis:")
print(f"  Time horizon: {T_horizon} year")
print(f"  Debt threshold: ${merton_config.debt_threshold:.2f}")
print(f"  Initial asset value: $1.00")
print(f"  Probability of default: {prob_default*100:.2f}%")

## 15. Save Trained Model

In [ ]:
# Save trained PINN model
model_path = 'trained_pinn_merton.pth'
solver.save(model_path)
print(f"\nModel saved to: {model_path}")

## Summary

In this notebook, we:

1. ✅ Configured the Merton structural credit risk model
2. ✅ Defined the Fokker-Planck equation for log-asset dynamics
3. ✅ Trained a Physics-Informed Neural Network to solve the PDE
4. ✅ Validated against analytical Gaussian solution
5. ✅ Extracted theoretical score functions via automatic differentiation
6. ✅ Demonstrated hybrid score blending for diffusion models
7. ✅ Computed default probabilities

### Next Steps

- Integrate with empirical diffusion models trained on market data
- Extend to multi-asset portfolios
- Apply to option pricing with stochastic volatility
- Calibrate to real credit spreads

---

**References**:
- PhD Research Document: Chapters 2-3
- Raissi et al. (2019): Physics-Informed Neural Networks
- Song et al. (2021): Score-Based Generative Modeling
- Merton (1974): On the Pricing of Corporate Debt